For the analysis of the data obtained from the different sources, we imported the following libraries

In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import os
from dotenv import load_dotenv
from sodapy import Socrata
import chromedriver_binary
from src.cleaning3 import *
import folium
from folium.plugins import HeatMapWithTime as HMWT
from folium.plugins import HeatMap as HM
from folium import plugins

We import the 12 CSVs containing NYC bike sharing data during 2019 and concatenate them into a single DataFrame (total).

In [2]:
Jan_2019= pd.read_csv("../DATA_original/201901-citibike-tripdata.csv")
Feb_2019=pd.read_csv("../DATA_original/201902-citibike-tripdata.csv")
Mar_2019=pd.read_csv("../DATA_original/201903-citibike-tripdata.csv")
Ap_2019=pd.read_csv("../DATA_original/201904-citibike-tripdata.csv")
May_2019=pd.read_csv("../DATA_original/201905-citibike-tripdata.csv")
Jun_2019=pd.read_csv("../DATA_original/201906-citibike-tripdata.csv")
Jul_2019=pd.read_csv("../DATA_original/201907-citibike-tripdata.csv")
Aug_2019=pd.read_csv("../DATA_original/201908-citibike-tripdata.csv")
Sept_2019=pd.read_csv("../DATA_original/201909-citibike-tripdata.csv")
Oct_2019=pd.read_csv("../DATA_original/201910-citibike-tripdata.csv")
Nov_2019=pd.read_csv("../DATA_original/201911-citibike-tripdata.csv")
Dec_2019=pd.read_csv("../DATA_original/201912-citibike-tripdata.csv")

In [3]:
total=pd.concat([Jan_2019,Feb_2019,Mar_2019,Ap_2019,May_2019,Jun_2019,Jul_2019,Aug_2019,Sept_2019,Oct_2019,Nov_2019,Dec_2019],axis=0).reset_index()

Once the data are in a single Df, we replace the values of the columns by data already unified and apply the function (location) to create a new column **"start_borough"** that will collect the neighborhoods to which the "start_stations" belong according to their latitude. 

In [4]:
COL=total.keys()
cols=[items.replace(" ","_").lower() for items in COL]

In [5]:
df1=total.set_axis(cols,axis=1)
copia=total.copy()

In [6]:
df1['start_borough']=df1['start_station_latitude'].apply(location)
df1.head(5)

,index,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender,start_borough
0,0,320,2019-01-01 00:01:47.4010,2019-01-01 00:07:07.5810,3160.0,Central Park West & W 76 St,40.778968,-73.973747,3283.0,W 89 St & Columbus Ave,40.788221,-73.970416,15839,Subscriber,1971,1,Manhattan
1,1,316,2019-01-01 00:04:43.7360,2019-01-01 00:10:00.6080,519.0,Pershing Square North,40.751873,-73.977706,518.0,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1,Manhattan
2,2,591,2019-01-01 00:06:03.9970,2019-01-01 00:15:55.4380,3171.0,Amsterdam Ave & W 82 St,40.785247,-73.976673,3154.0,E 77 St & 3 Ave,40.773142,-73.958562,27451,Subscriber,1987,1,Manhattan
3,3,2719,2019-01-01 00:07:03.5450,2019-01-01 00:52:22.6500,504.0,1 Ave & E 16 St,40.732219,-73.981656,3709.0,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,1,Manhattan
4,4,303,2019-01-01 00:07:35.9450,2019-01-01 00:12:39.5020,229.0,Great Jones St,40.727434,-73.993790,503.0,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,1,Manhattan


In [7]:
df1.to_csv('../DATA_pd/df1.csv')

With the data cleaned in the columns and using the locate function, we are going to create a dataframe only for December since we will use this month's data as a reference to establish the relationships with the other two dframes obtained by web scraping(Data_SQL jupyter notebook).
With respect to the gender column (Zero=unknown; 1=male; 2=female)

In [8]:
December=pd.DataFrame(Dec_2019).reset_index()
Dec=December.set_axis(cols,axis=1)
Dec['start_borough']=Dec['start_station_latitude'].apply(location)
Dec.head(5)

,index,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender,start_borough
0,0,602,2019-12-01 00:00:05.5640,2019-12-01 00:10:07.8180,3382,Carroll St & Smith St,40.680611,-73.994758,3304,6 Ave & 9 St,40.668127,-73.983776,41932,Subscriber,1970,1,Brooklyn
1,1,1206,2019-12-01 00:00:10.9630,2019-12-01 00:20:17.8820,362,Broadway & W 37 St,40.751726,-73.987535,500,Broadway & W 51 St,40.762288,-73.983362,18869,Customer,1999,1,Manhattan
2,2,723,2019-12-01 00:00:11.8180,2019-12-01 00:12:14.8310,146,Hudson St & Reade St,40.716250,-74.009106,238,Bank St & Washington St,40.736197,-74.008592,15334,Subscriber,1997,1,Manhattan
3,3,404,2019-12-01 00:00:12.2200,2019-12-01 00:06:56.8860,3834,Irving Ave & Halsey St,40.694670,-73.906630,3827,Halsey St & Broadway,40.685650,-73.915640,41692,Customer,1995,1,Queens
4,4,1059,2019-12-01 00:00:14.7230,2019-12-01 00:17:54.1860,500,Broadway & W 51 St,40.762288,-73.983362,3323,W 106 St & Central Park West,40.798186,-73.960591,40156,Subscriber,1961,1,Manhattan


In [9]:
Dec.to_csv('../DATA_pd/Dec.csv')

Based on the column "Start_station_name", through the groupby, we create a table that collects the most used stations, sorted by the number of trips in descending order. We will use **the top_10** (lat/lng) to set icons on the map, where the blue circle is the **top_1**, the most used (**"Pershing Square North - Manhattan**)

In [10]:
estaciones_salida=df1.groupby(["start_station_name","start_station_latitude","start_station_longitude"]).agg({"stoptime": "count"}).rename(columns={"stoptime":"trips"}).sort_values(by='trips', ascending=False)
estaciones_salida.head(10)

,,,trips
start_station_name,start_station_latitude,start_station_longitude,
Pershing Square North,40.751873,-73.977706,156574
E 17 St & Broadway,40.737050,-73.990093,121780
8 Ave & W 31 St,40.750585,-73.994685,119958
Broadway & E 22 St,40.740343,-73.989551,113138
Broadway & E 14 St,40.734546,-73.990741,113012
W 21 St & 6 Ave,40.741740,-73.994156,110305
West St & Chambers St,40.717548,-74.013221,105636
Broadway & W 60 St,40.769155,-73.981918,103167
Christopher St & Greenwich St,40.732916,-74.007114,101412


In [17]:
mapa=folium.Map(location=[40.751873, -73.977706],
                tiles='openstreetmap', zoom_start=12)

lat=[40.737050, 40.750585, 40.740343, 40.734546,40.741740,40.717548,40.769155,40.732916,40.732916]
lng=[-73.990093, -73.994685, -73.989551,-73.990741,-73.994156,-74.013221,-73.981918,-74.007114,-74.002777]

for i in range(len(lat)):
    folium.Marker([lat[i], lng[i]], icon=folium.Icon(color='red')).add_to(mapa)


In [18]:
folium.CircleMarker([40.751873, -73.977706], radius=10, icon=folium.Icon()).add_to(mapa) 
#top_1 station

In [19]:
mapa.save('../images/mapa.html')
mapa

In the following table, we grouped different values to see the most repeated trips, taking into account both the departure and arrival stations. 

In [20]:
top_trayectos=df1.groupby(["start_station_name","end_station_name",
                           "start_station_latitude",
                           "start_station_longitude",
                           "end_station_latitude",
                          "end_station_longitude"]).agg({"stoptime": "count"}).rename(columns={"stoptime":"trips"}).sort_values(by='trips', ascending=False)
top_trayectos.head(5)

trips
start_station_name              end_station_name                start_station_latitude start_station_longitude end_station_latitude end_station_longitude       
E 7 St & Avenue A               Cooper Square & Astor Pl        40.726218              -73.983799              40.729515            -73.990753              8426
Central Park S & 6 Ave          Central Park S & 6 Ave          40.765909              -73.976342              40.765909            -73.976342              6778
                                5 Ave & E 88 St                 40.765909              -73.976342              40.783070            -73.959390              6092
North Moore St & Greenwich St   Vesey Pl & River Terrace        40.720195              -74.010301              40.715338            -74.016584              5366
West Drive & Prospect Park West West Drive & Prospect Park West 40.661063              -73.979453              40.661063            -73.979453              5364

With the DataFrame containing all the records of 2019(df1) we grouped by the longitude and latitude data of the "start_station" to generate a **heat map** reflecting the use of bike sharing by neighborhoods during this period. To generate the map we started from the function "generateBaseMap()" in which the default location is the top_1 mentioned above.

In [21]:
df_copy = df1.copy()
df_copy['count'] = 1

df_copy[['start_station_latitude', 
         'start_station_longitude', 
         'count']].groupby(['start_station_latitude', 
                            'start_station_longitude']).sum().sort_values('count', 
                                                                   ascending=False).head(10)

,,count
start_station_latitude,start_station_longitude,
40.751873,-73.977706,156574
40.737050,-73.990093,121780
40.750585,-73.994685,119958
40.740343,-73.989551,113138
40.734546,-73.990741,113012
40.741740,-73.994156,110305
40.717548,-74.013221,105636
40.769155,-73.981918,103167
40.732916,-74.007114,101412


In [22]:
base_map = generateBaseMap() 

HM(data=df_copy[['start_station_latitude', 
                  'start_station_longitude', 
                  'count']].groupby(['start_station_latitude', 
                                     'start_station_longitude']).sum().reset_index().values.tolist(), 
    radius=8, 
    max_zoom=13).add_to(base_map) 
base_map.save('../images/base_heatmap.html')  

base_map   

### Web Scraping

**1. Open-Streets-Locations NYC**

Using **Selenium** we have scraped the web page containing different bike sharing data for New York City.
We are going to focus on the table representing those streets that became pedestrianized as a consequence of the Covid-19 pandemic.

New York City is opening 40 miles of streets to allow for greater social distancing during the COVID-19 crisis, with a plan to expand to a total of 100 miles. The streets are open daily from 8am-8pm (otherwise noted) for pedestrians and cyclists to use the roadbed of the streets while maintaining at least six feet of distance from others.

In [23]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time 
import urllib3

In [24]:
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False  
opciones.add_argument('--start-maximized')               
opciones.add_argument('--incognito')
opciones.add_argument("--no-sandbox")
opciones.add_argument("--disable-setuid-sandbox")

In [25]:
PATH= "./chromedriver.exe"
driver = webdriver.Chrome(PATH,options = opciones)
driver.get('https://data.cityofnewyork.us/Health/Open-Streets-Locations/uiay-nctu')

<ipython-input-25-be6f69a5c24e>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH,options = opciones)


In [26]:
tabla=driver.find_elements_by_tag_name("table")[5] #This is the table that collects the information we want to analyze.

<ipython-input-26-919741eeab1f>:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  tabla=driver.find_elements_by_tag_name("table")[5] #This is the table that collects the information we want to analyze.


We find that the table from which we want to extract the information is segmented into different sub-tables that we access by pressing the "Next" button that appears in all displays. That is why we implemented "button" in the code to be able to reach all the values. 

In [27]:
FINAL=[]
for i in range(0,21):
    tablita=driver.find_element_by_xpath("/html/body/main/div/div[1]/div/div[2]/section[3]/div[2]")
    tablita_ok=tablita.text.split('\n')
    boton = driver.find_element_by_xpath("//*[@id='app']/div/div[2]/section[3]/div[2]/div/div/div[4]/div[2]/span[1]/button[2]")
    boton.click()
    time.sleep(2)
    FINAL.append(tablita_ok)

<ipython-input-27-1585184e0a66>:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  tablita=driver.find_element_by_xpath("/html/body/main/div/div[1]/div/div[2]/section[3]/div[2]")
<ipython-input-27-1585184e0a66>:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  boton = driver.find_element_by_xpath("//*[@id='app']/div/div[2]/section[3]/div[2]/div/div/div[4]/div[2]/span[1]/button[2]")


When we have all the information of the table, we indicate the row that would be equivalent to the names of the columns ("**columnas**"). After, we clean up to eliminate those values that are being imported and that we do not need. For this cleaning we first use a for loop that will generate a list of strings with all the definitive values and to which we will apply the function **"split"** that divides the list "arr" in lists of 22 elements each one (each row of the original table).

In [28]:
columnas=FINAL[0][0:22]

In [29]:
arr=[]
for i in FINAL:
    for j in i:
        if j!='Y' and j!='Next'and j!='Previous'and 'Showing'not in j and 'Cool' not in j and j not in columnas and 'Fetching' not in j:
            arr.append(j)

In [30]:
filas=split(arr,22) #aplicamos la funcion

In [39]:
Open_Street_Location=pd.DataFrame(filas,columns=columnas)
Open_streets=Open_Street_Location.drop_duplicates()
Open_streets.head(5)

,Location or Partner,Borough,On Street,From Street,To Street,Type,Open Date,Monday_Start,Monday_End,Tuesday_Start,...,Wednesday_End,Thursday_Start,Thursday_End,Friday_Start,Friday_End,Saturday_Start,Saturday_End,Sunday_Start,Sunday_End,Day of Week
0,31st Ave Open Street Collective,Queens,31st Avenue,33rd Street,35th Street,Temporary Full Closure,07/24/2020,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,12:00PM,7:00PM,12:00PM,7:00PM,"S,U"
1,Sunnyside Woodside Open Streets,Queens,39th Avenue,45th Street,Woodside Avenue,Temporary Limited Local Access,05/14/2020,24/7,24/7,24/7,...,24/7,24/7,24/7,24/7,24/7,24/7,24/7,24/7,24/7,ALL
2,Bayside Village Business Improvement District,Queens,41st Avenue,Bell Boulevard,214th Place,Temporary Full Closure,03/12/2021,N/A,N/A,N/A,...,N/A,N/A,N/A,5:30PM,11:00PM,N/A,N/A,N/A,N/A,F
3,Third Avenue Business Improvement District,Bronx,Alexander Avenue,Bruckner Boulevard,East 134th Street,Temporary Full Closure,08/14/2020,N/A,N/A,N/A,...,N/A,2:00PM,8:00PM,2:00PM,8:00PM,11:00AM,8:00PM,N/A,N/A,"R,F,S"
4,Park Slope 5th Avenue Business Improvement Dis...,Brooklyn,4th Street,5th Avenue,4th Avenue,Temporary Limited Local Access,05/14/2020,10:00AM,6:00PM,10:00AM,...,6:00PM,10:00AM,6:00PM,10:00AM,6:00PM,10:00AM,6:00PM,N/A,N/A,"M,T,W,R,F,S"


Of the total number of streets that have been pedestrianized as a result of covid, we use groupby to count the number in each borough.After we will add this information inserting the column "Open_streets_perborough" to which we apply the function (total).

In [40]:
num_streets=Open_streets.groupby(["Borough"]).agg({"Type": "count"}).rename(columns={"Type":"total"}).sort_values(by='total', ascending=False)
num_streets

,total
Borough,
Manhattan,113
Brooklyn,77
Queens,35
Bronx,16
Staten Island,5


In [46]:
Open_streets["Open_streets_perborough"]=Open_streets["Borough"].apply(total)
Open_streets

<ipython-input-46-5398e596d36f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Open_streets["Open_streets_perborough"]=Open_streets["Borough"].apply(total)


,Location or Partner,Borough,On Street,From Street,To Street,Type,Open Date,Monday_Start,Monday_End,Tuesday_Start,...,Thursday_Start,Thursday_End,Friday_Start,Friday_End,Saturday_Start,Saturday_End,Sunday_Start,Sunday_End,Day of Week,Open_streets_perborough
0,31st Ave Open Street Collective,Queens,31st Avenue,33rd Street,35th Street,Temporary Full Closure,07/24/2020,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,12:00PM,7:00PM,12:00PM,7:00PM,"S,U",35
1,Sunnyside Woodside Open Streets,Queens,39th Avenue,45th Street,Woodside Avenue,Temporary Limited Local Access,05/14/2020,24/7,24/7,24/7,...,24/7,24/7,24/7,24/7,24/7,24/7,24/7,24/7,ALL,35
2,Bayside Village Business Improvement District,Queens,41st Avenue,Bell Boulevard,214th Place,Temporary Full Closure,03/12/2021,N/A,N/A,N/A,...,N/A,N/A,5:30PM,11:00PM,N/A,N/A,N/A,N/A,F,35
3,Third Avenue Business Improvement District,Bronx,Alexander Avenue,Bruckner Boulevard,East 134th Street,Temporary Full Closure,08/14/2020,N/A,N/A,N/A,...,2:00PM,8:00PM,2:00PM,8:00PM,11:00AM,8:00PM,N/A,N/A,"R,F,S",16
4,Park Slope 5th Avenue Business Improvement Dis...,Brooklyn,4th Street,5th Avenue,4th Avenue,Temporary Limited Local Access,05/14/2020,10:00AM,6:00PM,10:00AM,...,10:00AM,6:00PM,10:00AM,6:00PM,10:00AM,6:00PM,N/A,N/A,"M,T,W,R,F,S",77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,The Beacon School,Manhattan,West 44th Street,10th Avenue,11th Avenuye,Temporary Full Closure,11/16/2021,10:30AM,3:00PM,10:30AM,...,10:30AM,3:00PM,10:30AM,3:00PM,N/A,N/A,N/A,N/A,"M,T,W,R,F",113
242,City Managed / DOT,Bronx,Cromwell Avenue,McClellan Street,Jerome Avenue,Temporary Limited Local Access,11/16/2021,9:00AM,5:00PM,9:00AM,...,9:00AM,5:00PM,9:00AM,5:00PM,N/A,N/A,N/A,N/A,"M,T,W,R,F",16
243,City Managed / DOT,Manhattan,West 115th Street,5th Avenue,Lenox Avenue,Temporary Limited Local Access,11/16/2021,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,9:00AM,5:00PM,9:00AM,5:00PM,"S,U",113
244,City Managed / DOT,Brooklyn,Beverley Road,East 2nd Street,Church Avenue,Temporary Limited Local Access,11/16/2021,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,9:00AM,5:00PM,9:00AM,5:00PM,"S,U",77


In [47]:
Open_streets.to_csv('../DATA_pd/streets.csv')

**2. Boroughs of New York City - Demographic data**

First we use **beautiful soup** to get the name of the columns since they are not contained in the table body ('tbody') and clean the values.

In [48]:
url='https://en.wikipedia.org/wiki/Boroughs_of_New_York_City'
gdp=req.get(url).content
sopita=bs(gdp, 'html.parser')
tabla=sopita.find("table",{"class":"wikitable sortable"})
filas=tabla.find_all("tr")
filas_2=[i.text.split("\n") for i in filas]

In [49]:
columnas=filas_2[2][1:-1]
columnas2=[i.replace("billions(2012 US$)","billions gdp").lower() for i in columnas]

Then we use selenium to get directly the body of the table, the data I want to work with.

In [50]:
PATH= "./chromedriver.exe"
driver = webdriver.Chrome(PATH,options = opciones)
driver.get('https://en.wikipedia.org/wiki/Boroughs_of_New_York_City')

<ipython-input-50-e0a88c6266c4>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH,options = opciones)


In [51]:
tabla=driver.find_element_by_tag_name('tbody')
filas=tabla.find_elements_by_tag_name('tr')

<ipython-input-51-d73ff42e0f8c>:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  tabla=driver.find_element_by_tag_name('tbody')
/home/isabel11/anaconda3/envs/ironhack/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:359: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


In [52]:
data=[[i.text for i in f.find_elements_by_tag_name('td')] for f in filas]

In [53]:
BOROUGHS=pd.DataFrame(data,columns=columnas2)
BOROUGHS.dropna(axis=0, how='any',inplace=True)
BOROUGHS=BOROUGHS.drop(['county'],axis=1)
BOROUGHS

,borough,census (2020),billions gdp,square miles,squarekm,persons /mi2,persons /km2
0,The Bronx,"1,472,654",$ 42.695,42.2,109.3,"34,920","13,482"
1,Brooklyn,"2,736,074",$ 91.559,69.4,179.7,"39,438","15,227"
2,Manhattan,"1,694,251",$ 600.244,22.7,58.8,"74,781","28,872"
3,Queens,"2,405,464",$ 93.310,108.7,281.5,"22,125","8,542"
4,Staten Island,"495,747",$ 14.514,57.5,148.9,"8,618","3,327"


So that all the DataFrames have the same values of **"Borough "** we apply the names function that equals them with the other two dframes. 

In [54]:
BOROUGHS['borough']=BOROUGHS['borough'].apply(names) 
BOROUGHS

,borough,census (2020),billions gdp,square miles,squarekm,persons /mi2,persons /km2
0,Bronx,"1,472,654",$ 42.695,42.2,109.3,"34,920","13,482"
1,Brooklyn,"2,736,074",$ 91.559,69.4,179.7,"39,438","15,227"
2,Manhattan,"1,694,251",$ 600.244,22.7,58.8,"74,781","28,872"
3,Queens,"2,405,464",$ 93.310,108.7,281.5,"22,125","8,542"
4,Staten Island,"495,747",$ 14.514,57.5,148.9,"8,618","3,327"


Gross domestic product (GDP) is the monetary value of all finished goods and services made within a country during a specific period.
GDP provides an economic snapshot of a country, used to estimate the size of an economy and growth rate.
GDP can be calculated in three ways, using expenditures, production, or incomes. It can be adjusted for inflation and population to provide deeper insights.
Though it has limitations, GDP is a key tool to guide policy-makers, investors, and businesses in strategic decision-making.

In [55]:
BOROUGHS.to_csv('../DATA_pd/gdp.csv')